## MID TERM ASSIGNMENT


#### Author:Mounica Pothureddy

##### In this assignment, I am trying to implement a database in casssandra to store the songs information. It provides details about Song_ID, Title, Duration and the Genre of the song. Because of this it will be easy to store all songs in a single database and to retrive, modify and update the song information at any time. 

### Import packages

In [1]:
import pandas as pd
from cassandra.cluster import Cluster

### Create a session connection to Cassandra cluster

In [2]:
clstr = Cluster()
session = clstr.connect()

### Use session to 'talk' to cassandra

In [3]:
session.execute("DROP KEYSPACE IF EXISTS mid_term_mounica")

### Checking the existing keyspaces

In [4]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

bd23
system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema
w04python


### Create the keyspace and replication factor as 5. The replication factor is important as it replicates the data in other nodes and if there is any failure in master node the other nodes where copies of data is present help to get the data.

In [5]:
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS mid_term_mounica WITH REPLICATION = {
        'class':'SimpleStrategy', 
        'replication_factor':5
    }
""")

### So, here we can see that the new keyspace is created

In [21]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

bd23
mid_term_mounica
system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema
w04python


### Load data


#### Here, I am taking the Song_ID as the primary key to distinguish between each record. So, this column has unique values

In [11]:
session.execute("""
CREATE TABLE IF NOT EXISTS mid_term_mounica.song_db ( 
    Song_ID INT, 
    Title TEXT, 
    Duration INT, 
    Genre TEXT,
    PRIMARY KEY(Song_ID)
);
""")

### I used CSV to store the data and I am reading and storing it in df dataframe

In [7]:
df = pd.read_csv('Song_information.csv')
df

,Song_ID,Title,Duration,Genre
0,1,'SongTitle1',120,'pop'
1,2,'SongTitle2',140,'rock'
2,3,'SongTitle3',130,'classical'
3,4,'SongTitle4',110,'country'
4,5,'SongTitle5',160,'pop'


### Checking each row data

In [8]:
for index, row in df.iterrows():
    print(f" Song_ID= {row.Song_ID}, Title ={row.Title}, l_name = {row.Duration}, course = {row.Genre}")

 Song_ID= 1, Title ='SongTitle1', l_name = 120, course = 'pop'
 Song_ID= 2, Title ='SongTitle2', l_name = 140, course = 'rock'
 Song_ID= 3, Title ='SongTitle3', l_name = 130, course = 'classical'
 Song_ID= 4, Title ='SongTitle4', l_name = 110, course = 'country'
 Song_ID= 5, Title ='SongTitle5', l_name = 160, course = 'pop'


### Inserting into table

In [13]:
for index, row in df.iterrows():
    print(f"""
        INSERT INTO mid_term_mounica.song_db(Song_ID, Title, Duration, Genre)     
        VALUES ({row.Song_ID}, {row.Title}, {row.Duration}, {row.Genre});
        """
    )
    session.execute(f"""
        INSERT INTO mid_term_mounica.song_db(Song_ID, Title, Duration,Genre)     
        VALUES ({row.Song_ID}, {row.Title}, {row.Duration}, {row.Genre});
        """
    )


        INSERT INTO mid_term_mounica.song_db(Song_ID, Title, Duration, Genre)     
        VALUES (1, 'SongTitle1', 120, 'pop');
        

        INSERT INTO mid_term_mounica.song_db(Song_ID, Title, Duration, Genre)     
        VALUES (2, 'SongTitle2', 140, 'rock');
        

        INSERT INTO mid_term_mounica.song_db(Song_ID, Title, Duration, Genre)     
        VALUES (3, 'SongTitle3', 130, 'classical');
        

        INSERT INTO mid_term_mounica.song_db(Song_ID, Title, Duration, Genre)     
        VALUES (4, 'SongTitle4', 110, 'country');
        

        INSERT INTO mid_term_mounica.song_db(Song_ID, Title, Duration, Genre)     
        VALUES (5, 'SongTitle5', 160, 'pop');
        


### Now, I want to update a row in the database where Song_ID=2. I want to get the duration updated to 170 from 140. If we want to update information about any of the songs this actually helps.

In [15]:
session.execute(f"""UPDATE mid_term_mounica.song_db SET Duration=170 where Song_ID=2""")

### I am now checking if the data is updated or not in the database. 

In [17]:
row=session.execute("""select * from mid_term_mounica.song_db""");
for i in row:
    print(i)

Row(song_id=5, duration=160, genre='pop', title='SongTitle5')
Row(song_id=1, duration=120, genre='pop', title='SongTitle1')
Row(song_id=2, duration=170, genre='rock', title='SongTitle2')
Row(song_id=4, duration=110, genre='country', title='SongTitle4')
Row(song_id=3, duration=130, genre='classical', title='SongTitle3')


### Sometimes, we may need to remove some old data. So, here I am trying to delete a row of data where Song_ID=5

In [18]:
session.execute("""DELETE FROM mid_term_mounica.song_db where Song_ID=5""");

### Checking if that row of data is deleted or not

In [19]:
row=session.execute("""select * from mid_term_mounica.song_db""");
for i in row:
    print(i)

Row(song_id=1, duration=120, genre='pop', title='SongTitle1')
Row(song_id=2, duration=170, genre='rock', title='SongTitle2')
Row(song_id=4, duration=110, genre='country', title='SongTitle4')
Row(song_id=3, duration=130, genre='classical', title='SongTitle3')


### So, I have created a database with songs information with fields Song_ID, Title, Duration, Genre. In this way, we can create a database in cassandra and use the replications factor to store and retrive the data